[Reference](https://python.plainenglish.io/adding-ai-features-to-existing-applications-a-practical-tutorial-for-intermediate-developers-aa47f294e8a0)

# Setting Up Your Environment

In [1]:
# Create a virtual environment
!python -m venv .venv
!source .venv/bin/activate  # On Windows: ai_features_env\Scripts\activate

# Install required packages
!pip install transformers torch torchvision tensorflow-hub pillow requests flask

Error: Command '['/content/.venv/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: .venv/bin/activate: No such file or directory


# 1. Text Classification with HuggingFace Transformers

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

class NewsClassifier:
    def __init__(self):
        """
        Initialise the classifier with a pre-trained model.
        We're using a model specifically trained for news categorisation.
        """
        model_name = "facebook/bart-large-mnli"
        self.classifier = pipeline(
            "zero-shot-classification",
            model=model_name,
            tokenizer=model_name
        )

        # Define the categories we want to classify into
        self.categories = [
            "technology", "sports", "politics", "entertainment",
            "business", "health", "science"
        ]

    def classify_article(self, text, confidence_threshold=0.5):
        """
        Classify a news article into predefined categories.

        Args:
            text (str): The article text to classify
            confidence_threshold (float): Minimum confidence score to accept

        Returns:
            dict: Classification results with category and confidence
        """
        try:
            result = self.classifier(text, self.categories)

            # Extract the top prediction
            top_category = result['labels'][0]
            top_score = result['scores'][0]

            if top_score >= confidence_threshold:
                return {
                    'category': top_category,
                    'confidence': round(top_score, 3),
                    'all_scores': dict(zip(result['labels'], result['scores']))
                }
            else:
                return {
                    'category': 'unclassified',
                    'confidence': round(top_score, 3),
                    'reason': 'Low confidence score'
                }

        except Exception as e:
            return {'error': f"Classification failed: {str(e)}"}

# Usage example
if __name__ == "__main__":
    classifier = NewsClassifier()

    sample_article = """
    Apple announced today that its new iPhone will feature revolutionary
    AI capabilities powered by advanced machine learning chips. The technology
    promises to transform how users interact with their devices through
    natural language processing and computer vision features.
    """

    result = classifier.classify_article(sample_article)
    print(f"Article classified as: {result['category']} (confidence: {result['confidence']})")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Article classified as: technology (confidence: 0.764)


# 2. Image Recognition in Web Applications

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import requests
from io import BytesIO

class ImageRecogniser:
    def __init__(self):
        """
        Initialise with a pre-trained MobileNet model from TensorFlow Hub.
        MobileNet is optimised for mobile and web applications.
        """
        # Load the pre-trained model
        self.model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
        self.model = hub.load(self.model_url)

        # Load ImageNet labels (what the model can recognise)
        self.labels = self._load_imagenet_labels()

    def _load_imagenet_labels(self):
        """Load the 1000 ImageNet class labels."""
        labels_url = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"
        response = requests.get(labels_url)
        # First label is background, so we skip it
        return response.text.strip().split('\n')[1:]

    def preprocess_image(self, image_path_or_url):
        """
        Prepare an image for recognition by resizing and normalising.

        Args:
            image_path_or_url (str): Local path or URL to image

        Returns:
            tf.Tensor: Preprocessed image ready for prediction
        """
        try:
            if image_path_or_url.startswith('http'):
                # Handle URL
                response = requests.get(image_path_or_url)
                image = Image.open(BytesIO(response.content))
            else:
                # Handle local file
                image = Image.open(image_path_or_url)

            # Convert to RGB if necessary
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Resize to model's expected input size
            image = image.resize((224, 224))

            # Convert to numpy array and normalise
            image_array = np.array(image) / 255.0

            # Add batch dimension
            image_batch = np.expand_dims(image_array, axis=0)

            return tf.convert_to_tensor(image_batch, dtype=tf.float32)

        except Exception as e:
            raise ValueError(f"Failed to process image: {str(e)}")

    def recognise_image(self, image_path_or_url, top_k=5):
        """
        Identify objects in an image and return top predictions.

        Args:
            image_path_or_url (str): Path or URL to the image
            top_k (int): Number of top predictions to return

        Returns:
            list: Top predictions with labels and confidence scores
        """
        try:
            # Preprocess the image
            processed_image = self.preprocess_image(image_path_or_url)

            # Make prediction
            predictions = self.model(processed_image)

            # Get probabilities
            probabilities = tf.nn.softmax(predictions[0]).numpy()

            # Get top k predictions
            top_indices = np.argsort(probabilities)[-top_k:][::-1]

            results = []
            for idx in top_indices:
                results.append({
                    'label': self.labels[idx],
                    'confidence': round(float(probabilities[idx]), 4),
                    'percentage': round(float(probabilities[idx]) * 100, 2)
                })

            return results

        except Exception as e:
            return {'error': f"Recognition failed: {str(e)}"}

# Flask integration example
from flask import Flask, request, jsonify, render_template_string

app = Flask(__name__)
recogniser = ImageRecogniser()

@app.route('/recognise', methods=['POST'])
def recognise_endpoint():
    """API endpoint for image recognition."""
    try:
        if 'image_url' in request.json:
            image_source = request.json['image_url']
        elif 'file' in request.files:
            # Handle file upload (save temporarily)
            file = request.files['file']
            image_source = f"/tmp/{file.filename}"
            file.save(image_source)
        else:
            return jsonify({'error': 'No image provided'}), 400

        results = recogniser.recognise_image(image_source)
        return jsonify(results)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == "__main__":
    # Test the recogniser
    test_url = "https://images.unsplash.com/photo-1574158622682-e40e69881006?w=500"
    results = recogniser.recognise_image(test_url)

    print("Top 5 predictions:")
    for result in results:
        print(f"  {result['label']}: {result['percentage']}%")

Top 5 predictions:
  tiger cat: 36.82%
  cougar: 29.33%
  Persian cat: 20.8%
  leopard: 0.67%
  letter opener: 0.22%


# 3. Sentiment Analysis for Product Reviews

In [4]:
from transformers import pipeline
import re
from datetime import datetime

class SentimentAnalyser:
    def __init__(self):
        """
        Initialise with a robust sentiment analysis model.
        We're using a model trained specifically for product reviews.
        """
        self.sentiment_pipeline = pipeline(
            "sentiment-analysis",
            model="cardiffnlp/twitter-roberta-base-sentiment-latest",
            tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest"
        )

        # Define emotion categories for more nuanced analysis
        self.emotion_pipeline = pipeline(
            "text-classification",
            model="j-hartmann/emotion-english-distilroberta-base",
            tokenizer="j-hartmann/emotion-english-distilroberta-base"
        )

    def clean_text(self, text):
        """
        Clean and prepare text for analysis.

        Args:
            text (str): Raw review text

        Returns:
            str: Cleaned text ready for analysis
        """
        # Remove extra whitespace and newlines
        text = re.sub(r'\s+', ' ', text.strip())

        # Remove HTML tags if present
        text = re.sub(r'<[^>]+>', '', text)

        # Handle common contractions for better analysis
        contractions = {
            "won't": "will not", "can't": "cannot", "n't": " not",
            "'re": " are", "'ve": " have", "'ll": " will", "'d": " would"
        }

        for contraction, expansion in contractions.items():
            text = text.replace(contraction, expansion)

        return text

    def analyse_sentiment(self, text):
        """
        Perform comprehensive sentiment analysis.

        Args:
            text (str): Review or feedback text

        Returns:
            dict: Detailed sentiment analysis results
        """
        try:
            # Clean the input text
            cleaned_text = self.clean_text(text)

            # Basic sentiment analysis
            sentiment_result = self.sentiment_pipeline(cleaned_text)[0]

            # Emotion analysis for deeper insights
            emotion_result = self.emotion_pipeline(cleaned_text)[0]

            # Determine overall sentiment category
            sentiment_label = sentiment_result['label'].lower()
            if sentiment_label in ['positive', 'label_2']:  # Some models use different labels
                overall_sentiment = 'positive'
            elif sentiment_label in ['negative', 'label_0']:
                overall_sentiment = 'negative'
            else:
                overall_sentiment = 'neutral'

            # Calculate confidence and provide interpretation
            confidence = sentiment_result['score']

            interpretation = self._interpret_results(
                overall_sentiment, confidence, emotion_result['label']
            )

            return {
                'overall_sentiment': overall_sentiment,
                'confidence': round(confidence, 3),
                'primary_emotion': emotion_result['label'],
                'emotion_confidence': round(emotion_result['score'], 3),
                'interpretation': interpretation,
                'timestamp': datetime.now().isoformat(),
                'processed_text_length': len(cleaned_text.split())
            }

        except Exception as e:
            return {'error': f"Analysis failed: {str(e)}"}

    def _interpret_results(self, sentiment, confidence, emotion):
        """
        Provide human-readable interpretation of results.

        Args:
            sentiment (str): Overall sentiment
            confidence (float): Confidence score
            emotion (str): Primary emotion detected

        Returns:
            str: Human-readable interpretation
        """
        confidence_level = "high" if confidence > 0.8 else "moderate" if confidence > 0.6 else "low"

        interpretations = {
            'positive': {
                'joy': f"Customer expresses clear satisfaction with {confidence_level} confidence",
                'surprise': f"Customer seems pleasantly surprised with {confidence_level} confidence",
                'trust': f"Customer shows strong confidence in the product with {confidence_level} confidence"
            },
            'negative': {
                'anger': f"Customer expresses frustration with {confidence_level} confidence",
                'disgust': f"Customer shows strong dissatisfaction with {confidence_level} confidence",
                'fear': f"Customer has concerns or worries with {confidence_level} confidence",
                'sadness': f"Customer expresses disappointment with {confidence_level} confidence"
            },
            'neutral': {
                'neutral': f"Customer provides balanced feedback with {confidence_level} confidence"
            }
        }

        return interpretations.get(sentiment, {}).get(emotion,
            f"Customer sentiment is {sentiment} with {confidence_level} confidence")

    def batch_analyse(self, reviews_list):
        """
        Analyse multiple reviews efficiently.

        Args:
            reviews_list (list): List of review texts

        Returns:
            dict: Aggregated analysis results
        """
        results = []
        sentiment_counts = {'positive': 0, 'negative': 0, 'neutral': 0}
        emotion_counts = {}

        for review in reviews_list:
            analysis = self.analyse_sentiment(review)
            if 'error' not in analysis:
                results.append(analysis)
                sentiment_counts[analysis['overall_sentiment']] += 1
                emotion = analysis['primary_emotion']
                emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1

        total_reviews = len(results)
        if total_reviews == 0:
            return {'error': 'No valid reviews processed'}

        # Calculate percentages
        sentiment_percentages = {
            k: round((v / total_reviews) * 100, 1)
            for k, v in sentiment_counts.items()
        }

        return {
            'total_reviews': total_reviews,
            'sentiment_breakdown': sentiment_percentages,
            'top_emotions': sorted(emotion_counts.items(), key=lambda x: x[1], reverse=True)[:5],
            'individual_results': results,
            'summary': f"{sentiment_percentages['positive']}% positive, "
                      f"{sentiment_percentages['negative']}% negative, "
                      f"{sentiment_percentages['neutral']}% neutral"
        }

# Practical integration example for a product review form
class ReviewFormIntegration:
    def __init__(self):
        self.analyser = SentimentAnalyser()

    def process_review_submission(self, review_data):
        """
        Process a review submission with sentiment analysis.

        Args:
            review_data (dict): Review form data

        Returns:
            dict: Enhanced review data with sentiment insights
        """
        review_text = review_data.get('review_text', '')

        if not review_text.strip():
            return {'error': 'Review text is required'}

        # Perform sentiment analysis
        sentiment_analysis = self.analyser.analyse_sentiment(review_text)

        # Enhance the review data
        enhanced_review = {
            **review_data,
            'sentiment_analysis': sentiment_analysis,
            'requires_attention': (
                sentiment_analysis.get('overall_sentiment') == 'negative' and
                sentiment_analysis.get('confidence', 0) > 0.7
            ),
            'processing_timestamp': datetime.now().isoformat()
        }

        # Add automated response suggestions based on sentiment
        if sentiment_analysis.get('overall_sentiment') == 'negative':
            enhanced_review['suggested_action'] = 'Consider reaching out for customer support'
        elif sentiment_analysis.get('overall_sentiment') == 'positive':
            enhanced_review['suggested_action'] = 'Consider featuring in testimonials'

        return enhanced_review

# Usage demonstration
if __name__ == "__main__":
    analyser = SentimentAnalyser()

    # Single review analysis
    sample_review = """
    I've been using this product for three months now and I'm really impressed!
    The quality is excellent and it solved exactly the problem I was having.
    Customer service was also very helpful when I had questions. Highly recommend!
    """

    result = analyser.analyse_sentiment(sample_review)
    print(f"Sentiment: {result['overall_sentiment']} ({result['confidence']})")
    print(f"Primary emotion: {result['primary_emotion']}")
    print(f"Interpretation: {result['interpretation']}")

    # Batch analysis example
    reviews = [
        "Great product, works perfectly!",
        "Terrible quality, broke after one week.",
        "It's okay, nothing special but does the job.",
        "Amazing! Exceeded my expectations completely!",
        "Poor customer service, very disappointed."
    ]

    batch_results = analyser.batch_analyse(reviews)
    print(f"\nBatch Analysis Summary: {batch_results['summary']}")

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


Sentiment: positive (0.984)
Primary emotion: surprise
Interpretation: Customer seems pleasantly surprised with high confidence

Batch Analysis Summary: 60.0% positive, 40.0% negative, 0.0% neutral


# Bringing It All Together: Complete Integration Example


In [5]:
from flask import Flask, request, jsonify, render_template_string
import json

app = Flask(__name__)

# Initialise all our AI components
news_classifier = NewsClassifier()
image_recogniser = ImageRecogniser()
sentiment_analyser = SentimentAnalyser()

@app.route('/ai-dashboard', methods=['GET', 'POST'])
def ai_dashboard():
    """
    A simple dashboard demonstrating all AI features.
    """
    if request.method == 'GET':
        return render_template_string(DASHBOARD_TEMPLATE)

    # Handle different AI requests
    feature = request.json.get('feature')

    if feature == 'text_classification':
        text = request.json.get('text', '')
        result = news_classifier.classify_article(text)
        return jsonify(result)

    elif feature == 'image_recognition':
        image_url = request.json.get('image_url', '')
        result = image_recogniser.recognise_image(image_url)
        return jsonify(result)

    elif feature == 'sentiment_analysis':
        text = request.json.get('text', '')
        result = sentiment_analyser.analyse_sentiment(text)
        return jsonify(result)

    else:
        return jsonify({'error': 'Unknown feature requested'}), 400

# Simple HTML template for testing
DASHBOARD_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>AI Features Dashboard</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        .feature-section { border: 1px solid #ddd; margin: 20px 0; padding: 20px; }
        button { background-color: #007bff; color: white; padding: 10px 20px; border: none; cursor: pointer; }
        textarea, input { width: 100%; padding: 10px; margin: 10px 0; }
        .result { background-color: #f8f9fa; padding: 15px; margin-top: 10px; border-radius: 5px; }
    </style>
</head>
<body>
    <h1>AI Features Integration Demo</h1>

    <div class="feature-section">
        <h2>Text Classification</h2>
        <textarea id="classifyText" placeholder="Enter news article text here..."></textarea>
        <button onclick="classifyText()">Classify Text</button>
        <div id="classifyResult" class="result"></div>
    </div>

    <div class="feature-section">
        <h2>Image Recognition</h2>
        <input type="url" id="imageUrl" placeholder="Enter image URL here...">
        <button onclick="recogniseImage()">Recognise Image</button>
        <div id="imageResult" class="result"></div>
    </div>

    <div class="feature-section">
        <h2>Sentiment Analysis</h2>
        <textarea id="sentimentText" placeholder="Enter review or feedback text here..."></textarea>
        <button onclick="analyseSentiment()">Analyse Sentiment</button>
        <div id="sentimentResult" class="result"></div>
    </div>

    <script>
        async function makeRequest(feature, data) {
            const response = await fetch('/ai-dashboard', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({feature, ...data})
            });
            return await response.json();
        }

        async function classifyText() {
            const text = document.getElementById('classifyText').value;
            const result = await makeRequest('text_classification', {text});
            document.getElementById('classifyResult').innerHTML =
                `<strong>Category:</strong> ${result.category}<br>
                 <strong>Confidence:</strong> ${result.confidence}`;
        }

        async function recogniseImage() {
            const imageUrl = document.getElementById('imageUrl').value;
            const result = await makeRequest('image_recognition', {image_url: imageUrl});
            let html = '<strong>Top Predictions:</strong><br>';
            result.forEach(item => {
                html += `${item.label}: ${item.percentage}%<br>`;
            });
            document.getElementById('imageResult').innerHTML = html;
        }

        async function analyseSentiment() {
            const text = document.getElementById('sentimentText').value;
            const result = await makeRequest('sentiment_analysis', {text});
            document.getElementById('sentimentResult').innerHTML =
                `<strong>Sentiment:</strong> ${result.overall_sentiment}<br>
                 <strong>Emotion:</strong> ${result.primary_emotion}<br>
                 <strong>Confidence:</strong> ${result.confidence}<br>
                 <strong>Interpretation:</strong> ${result.interpretation}`;
        }
    </script>
</body>
</html>
'''

if __name__ == '__main__':
    app.run(debug=True)